In [2]:
import json
import re

In [3]:
split = 'train'

In [6]:
with open(f'anli1_{split}_llama3.json') as f:
    llama_output = json.load(f)

In [5]:
with open(f'anli1_{split}_matches.txt', 'r') as f:
    matches = f.readlines()

In [ ]:
with open(f'anli1_{split}_random_llama3.json') as f:
    llama_output = json.load(f)
    f.close()
with open(f'anli1_{split}_matches.txt', 'r') as f:
    matches = f.readlines()
    f.close()

In [28]:
print(curate(llama_output, 2))

('alexandra bastedo was a vegetarian and animal welfare advocate,', 'neutral')


In [27]:
def curate(data, index):
    
    # Extracting fields
    gen_label = data[index]['gen_label'].lower().replace('rationale:', '').strip()
    org_label = data[index]['org_label']

    # Splitting rationale and label
    if ('the answer is' in gen_label and len(gen_label) > 1):
        try:
            rationale, label = gen_label.split('the answer is')
        except:
            rationale = gen_label.split('the answer is')[0]
            label = ""
    else:
        rationale = gen_label
        label = ""

    # Preparing the rationale
    if rationale == "":
        rationale = gen_label.replace("the answer is","").replace('answer:', '').strip()    
    rationale = rationale.replace('hence', '').strip().strip('.')

    # Preparing the label
    if ('(a)' in  gen_label or 'entailment' in gen_label):
        label = 'entailment'
    elif ('(b)' in  gen_label or 'neutral' in gen_label):
        label = 'neutral'
    elif ('(c)' in  gen_label or 'contradiction' in gen_label):
        label = 'contradiction'
    else:
        label = org_label
    
    return rationale, label

In [29]:
file_num = 0
dumper = []

for i in range(len(llama_output)):
    
    rationale, label = curate(llama_output, i)

    matched_num = int(matches[i].split()[0])
    rag_rationale, rag_label = curate(llama_output, matched_num)
    rag_question = llama_output[matched_num]['input']
    rag_answer = f"{rag_rationale}. So the answer is {rag_label}"
    
    entry = f"{rationale} So the answer is {label} {rag_question} A: {rag_answer}"
    # print(entry)
    # if i == 5:
    #     break
    dumper.append(entry)
    
  
    if i % 5000 == 4999 or i == (len(llama_output) - 1):
        with open(f"./llm/{split}_CoT_{file_num}.json", "w") as f:
            json.dump(dumper, f)
        dumper = []
        file_num += 1

the system presently comprises four urban routes,. So the answer is neutral premise: The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In operation since 1953, the system presently comprises four urban routes., hypothesis: Since 1953 the Parma Troleybus system has been comprised of four urban routes. A: the premise explicitly states that the system presently comprises four urban routes, and the hypothesis confirms this information,. So the answer is entailment
alexandra lendon bastedo was a british actress who acted in the 1968 british espionage/science fiction adventure series "the champions". she was best known for her role as secret agent sharron macready. she was cited as a sex symbol of the 1960s and 1970s. bastedo was a vegetarian and animal welfare advocate. the champions was a fact-based series. the role of sharron macready was played b